In [10]:
import ee
import geemap
# Trigger the authentication flow.


# Initialize the library.
ee.Initialize()

In [11]:
Map = geemap.Map()

admin = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1");
maharashtra = admin.filter(ee.Filter.eq('ADM1_NAME', 'West Bengal'))
geometry = maharashtra.geometry()

point = ee.Geometry.Point([80.3288, 23.5121])

image = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterDate('2019-01-01', '2023-12-31') \
    .sort('CLOUD_COVER') \
    .min() \
    .select('B[1-7]') \
    .clip(geometry)


vis_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B5', 'B4', 'B3']
}

Map.centerObject(point, 5)
Map.addLayer(image, vis_params, "Landsat-8")

#Map

In [12]:
modis = ee.ImageCollection("MODIS/061/MCD12Q1") \
              .filterDate('2019-01-01', '2023-01-01') \
              .select('LC_Type1') \
              .reduce(ee.Reducer.median())

points = modis.sample(**{
    'region': ee.Geometry.Rectangle([72.0000, 23.0000, 82.0000, 15.0000]),
    'scale': 30,
    'numPixels': 5000,
    'seed': 0,
    'geometries': True  # Set this to False to ignore geometries
})

Map.addLayer(points, {}, 'training', False)
Map

Map(center=[23.512099999999997, 80.3288], controls=(WidgetControl(options=['position', 'transparent_bg'], widg…

In [13]:
print(points.size().getInfo())

5000


In [14]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [75.96063229451673, 21.19164441922813]}, 'id': '0', 'properties': {'LC_Type1_median': 12}}


In [15]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']


# This property of the table stores the land cover labels.
label = 'LC_Type1_median'
input = image.select(bands)

# Overlay the points on the imagery to get training.
training = input.sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileCart().train(training, label, bands)

print(training.first().getInfo())

None


In [16]:
result = image.select(bands).classify(trained)

# Display the clusters with random colors.
# Map.addLayer(result.randomVisualizer(), {}, 'classfied')
# Map

In [17]:
map2 = geemap.Map()

ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""
#############################################################
landcover = ee.ImageCollection("MODIS/061/MCD12Q1") \
              .filterDate('2019-01-01', '2023-12-31') \
              .select('LC_Type1') \
              .reduce(ee.Reducer.mean()) \
              .clip(geometry)

LandCoverVis = {
  'min': 1.0,
  'max': 17.0,
  'palette': [
    '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159',
    'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c',
    '69fff8', 'f9ffa4', '1c0dff'
  ],
}

Map.centerObject(point, 5)
Map.addLayer(landcover, LandCoverVis, 'MODIS Land Cover')

legend_dict = geemap.legend_from_ee(ee_class_table)
#map2.add_legend(legend_title="MODIS Global Land Cover", legend_dict=legend_dict)
#map2

In [18]:
landsatcover = result.set('classification_class_values', [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16])
landsatcover = landsatcover.set('classification_class_palette', ['1c0dff', '05450a', '086a10', '54a708', '78d203', '009900', 'c6b044', 'dcd159', 'dade48', 'fbff13', 'b6ff05', '27ff87', 'c24f44', 'a5a5a5', 'ff6d4c', '69fff8', 'f9ffa4',   ])
Map.addLayer(landsatcover, {}, 'Land cover')

EEException: No valid training data were found.

In [ ]:
ee_class_table = """

Value	Color	Description
0	1c0dff	Water
1	05450a	Evergreen needleleaf forest
2	086a10	Evergreen broadleaf forest
3	54a708	Deciduous needleleaf forest
4	78d203	Deciduous broadleaf forest
5	009900	Mixed forest
6	c6b044	Closed shrublands
7	dcd159	Open shrublands
8	dade48	Woody savannas
9	fbff13	Savannas
10	b6ff05	Grasslands
11	27ff87	Permanent wetlands
12	c24f44	Croplands
13	a5a5a5	Urban and built-up
14	ff6d4c	Cropland/natural vegetation mosaic
15	69fff8	Snow and ice
16	f9ffa4	Barren or sparsely vegetated
254	ffffff	Unclassified

"""

legend_dict = geemap.legend_from_ee(ee_class_table)
Map.add_legend(legend_title="Land Cover Legend", legend_dict=legend_dict)

Map